In [56]:
import parser
import numpy as np
import material_analytics
import graph_suite as plot
reload(plot)

<module 'graph_suite' from 'graph_suite.pyc'>

In [85]:
data = parser.stress_strain('ref/1150.dat').get_experimental_data()

def yield_classic(data, cutoff = 0.001, numpts = 1001):
    
    startpt = cutoff # can't be zero or log will freak out
    
    data = material_analytics.log_prep(data,cutoff = cutoff)
    bestfit = material_analytics.log_approx(data)
    domain = np.linspace(startpt,max(data[:,0]),numpts)
    vals = material_analytics.combine_data(domain,bestfit(domain))
    deriv1st = material_analytics.get_slopes(material_analytics.combine_data(domain,bestfit(domain)))

    find_nearest = lambda array,value: array[(np.abs(array-value)).argmin()]

    ave_slope = (vals[-1,1]-vals[0,1])/(vals[-1,0]-vals[0,0])
    index_ave = np.where(deriv1st == find_nearest(deriv1st, ave_slope))
    log_bend = vals[index_ave][0]
    young_mod = (log_bend[1]-vals[0,1])/(log_bend[0]-vals[0,0])

    linear_mod_2 = lambda x: young_mod*(x-0.2) + vals[0,1] # we will find where this intersects our exp data
    samples2 = material_analytics.samplepoints(linear_mod_2,[startpt,max(vals[:,0])],numpts)

    diff = np.abs(samples2[:,1] - vals[:,1])

    """We can call this the intersection, its the closest point"""
    leastindex = diff.argmin()
    cross_est = vals[leastindex].reshape(1,-1)[0,1]
    yieldstress = find_nearest(data[:,1].reshape(-1,1),cross_est)[0]
    yieldpt = data[np.where(data[:,1] == yieldstress)]
    return yieldpt[0]

plot.plotmult2D(data,yield_classic(data)[None,:],
                title = 'Stress vs Strain', xtitle = "Strain ($\epsilon$)", ytitle= "Stress ($\sigma$)")